In [1]:
from preamble import *

In [2]:
file_path = './data/d_pressure_flux.csv'

In [3]:
data = pd.read_csv(file_path)
data['datetime'] = pd.to_datetime(data['datetime'])
data.head()

,datetime,pressure,flux
0,2023-01-01 00:01:00,9.69,75.23
1,2023-01-01 00:02:00,9.69,74.65
2,2023-01-01 00:03:00,9.69,75.28
3,2023-01-01 00:04:00,9.69,74.67
4,2023-01-01 00:05:00,9.69,75.06


In [4]:
data_one_day = data[:1439]
data_one_day.tail()

,datetime,pressure,flux
1434,2023-01-01 23:55:00,9.49,143.97
1435,2023-01-01 23:56:00,9.49,145.10
1436,2023-01-01 23:57:00,9.49,144.48
1437,2023-01-01 23:58:00,9.49,145.45
1438,2023-01-01 23:59:00,9.49,144.39


In [5]:
# 송수 압력 P (단위: Pa)
# 유량 Q (단위: m³/min)
# 물의 밀도 ρ (약 1000 kg/m³)
# 중력가속도 g (9.81 m/s²)
# 펌프 효율 eta (소수로 표현)

def calculate_power_from_pressure(P, Q_m3_min, eta):
    rho = 1000  # 물의 밀도 (kg/m³)
    g = 9.81    # 중력가속도 (m/s²)

    # 유량을 m³/min에서 m³/s로 변환
    Q_m3_s = Q_m3_min / 60.0

    # 전력 소비량 계산 (킬로와트)
    power_kW = (Q_m3_s * P * 100000) / (eta * 1000)
    print(power_kW)
    return power_kW

P = 2  # 송수 압력 (bar) 가정
Q_m3_min = 32 / 60  # m³/min
eta = 0.85  # 효율

power_kW = calculate_power_from_pressure(P, Q_m3_min, eta)
print(f"전력 소비량: {power_kW} kW")

2.0915032679738563
전력 소비량: 2.0915032679738563 kW


In [6]:
import pickle

with open('best_polynomial_regression_model.pkl', 'rb') as f:
  model, poly = pickle.load(f)

In [7]:
from datetime import datetime, time

# 전기 요금 정보 (인상된 요금 반영)
electricity_rates = {
    'summer': {
        'off_peak': 64.37,  # 경부하 요금 (원/kWh)
        'mid_peak': 92.46,  # 중간부하 요금 (원/kWh)
        'on_peak': 123.88   # 최대부하 요금 (원/kWh)
    },
    'spring_fall': {
        'off_peak': 64.37,  # 경부하 요금 (원/kWh)
        'mid_peak': 69.50,  # 중간부하 요금 (원/kWh)
        'on_peak': 86.88    # 최대부하 요금 (원/kWh)
    },
    'winter': {
        'off_peak': 71.88,  # 경부하 요금 (원/kWh)
        'mid_peak': 90.80,  # 중간부하 요금 (원/kWh)
        'on_peak': 116.47   # 최대부하 요금 (원/kWh)
    }
}

# 시간대 구분
time_periods = {
    'summer': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
        'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
    },
    'spring_fall': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
        'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
    },
    'winter': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(10, 0)), (time(12, 0), time(17, 0)), (time(20, 0), time(22, 0))],
        'on_peak': [(time(10, 0), time(12, 0)), (time(17, 0), time(20, 0)), (time(22, 0), time(23, 0))]
    }
}

In [8]:
for _, row in data.iterrows():
  print(row['flux'])
  break

75.22504


In [9]:
def calculate_daily_cost_by_linear(data):
  # data = pd.read_csv(file_path, parse_dates=['datetime'])
  total_cost = 0
  poly = PolynomialFeatures(3)
  for _, row in data.iterrows():
    season, period = get_season_and_period(row['datetime'])
    flux_poly = poly.fit_transform(np.array([[row['flux']]]))
    power_kW = model.predict(flux_poly)[0]
    cost_per_kWh = electricity_rates[season][period]
    energy_kWh = power_kW / 60  # converting kW to kWh per minute
    total_cost += energy_kWh * cost_per_kWh
    print(power_kW)
  return total_cost

In [10]:
def get_season_and_period(dt):
  month = dt.month
  current_time = dt.time()
  
  if month in [7, 8]:
    season = 'summer'
  elif month in [11, 12, 1, 2]:
    season = 'winter'
  else:
    season = 'spring_fall'
  
  for period, times in time_periods[season].items():
    for start, end in times:
      if start <= current_time < end:
        return season, period
  return season, 'off_peak'

def calculate_daily_cost(data, eta = 0.85):
  # data = pd.read_csv(file_path, parse_dates=['datetime'])
  total_cost = 0
  
  for _, row in data.iterrows():
    season, period = get_season_and_period(row['datetime'])
    power_kW = calculate_power_from_pressure(row['pressure'], row['flux'], eta)
    cost_per_kWh = electricity_rates[season][period]
    energy_kWh = power_kW / 60  # converting kW to kWh per minute
    total_cost += energy_kWh * cost_per_kWh
    
  return total_cost

eta = 0.85
cost = int(calculate_daily_cost(data_one_day, eta))
cost_format = format(cost, ',')
print(f'Total daily electricity cost: {cost_format} KRW')


1428.814822254902
1418.5351588848039
1429.7728731862746
1418.369044669118
1425.786875
1416.9282041421568
1419.1862683823529
1420.9257937500001
1419.2656643382354
1420.628713909314
1429.467968848039
1416.3179326470588
1422.869930085784
1421.8527728553925
1415.9581455392156
1424.9030201102942
1413.580166862745
1418.4681074509806
1421.3522775980389
1424.4483839215686
1419.8840523039219
1425.6326348039217
1418.83385814951
1408.9267320955882
1423.676775367647
1415.526504534314
1422.9046695588236
1416.4105012867647
1419.4273940931373
1419.7660112254903
1424.8168963235294
1421.3021998284314
1424.5632034803918
1430.2498163235298
1421.0533200000002
1421.2316813848038
1410.2097209313727
1418.1816624264707
1415.5371556495097
1422.3329699999997
1421.9717098529413
1424.17806
1417.8521288357842
1412.563388272059
1433.1380594852942
1427.9797471446082
1428.962581519608
1449.7972557352941
1862.2549660539214
1839.8335096323528
1849.585316360294
1841.6692852205879
1841.0663682598038
1838.937438627451
184